# Explore the unemployment and income data from USDA

In [40]:
import pandas as pd

df = pd.read_csv("https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/Unemployment2023.csv?v=67344")
df.head(-10)

,FIPS_Code,State,Area_Name,Attribute,Value
0,0,US,United States,Civilian_labor_force_2000,142601576.0
1,0,US,United States,Employed_2000,136904853.0
2,0,US,United States,Unemployed_2000,5696723.0
3,0,US,United States,Unemployment_rate_2000,4.0
4,0,US,United States,Civilian_labor_force_2001,143786537.0
...,...,...,...,...,...
329711,72153,PR,"Yauco Municipio, PR",Employed_2019,8409.0
329712,72153,PR,"Yauco Municipio, PR",Unemployed_2019,1442.0
329713,72153,PR,"Yauco Municipio, PR",Unemployment_rate_2019,14.6
329714,72153,PR,"Yauco Municipio, PR",Civilian_labor_force_2021,10205.0


## Clean the data

In [ ]:
# select only data in housing data
# for exploration purposes only
housing = pd.read_csv("/workspaces/greener/static/county_score/home_values_county.csv")

# Ensure StateCodeFIPS is in two digit format and MunicipalCodeFIPS is in 3 digit format.

housing["StateCodeFIPS"] = housing["StateCodeFIPS"].apply(lambda x: str(x).zfill(2))
housing["MunicipalCodeFIPS"] = housing['MunicipalCodeFIPS'].apply(lambda x: str(x).zfill(3))

# Combine State and Municipal FIPS codes to get a 5 digit FIPS code.
housing.insert(0,"fips",(housing["StateCodeFIPS"] + housing["MunicipalCodeFIPS"])) 
housing["fips"]= housing["StateCodeFIPS"] + housing["MunicipalCodeFIPS"]
# create list of codes to filter by
codes = housing['fips'].unique()
print(len(codes))
# format fips codes in df to be 5 digits
df['FIPS_Code'] = df['FIPS_Code'].apply(lambda x: str(x).zfill(5))


df = df[df["FIPS_Code"].isin(codes)]
print(len(df['FIPS_Code'].unique()))
df.head()

3073
3073


,FIPS_Code,State,Area_Name,Attribute,Value
195,01001,AL,"Autauga County, AL",Rural_Urban_Continuum_Code_2023,2.0
196,01001,AL,"Autauga County, AL",Urban_Influence_Code_2013,2.0
197,01001,AL,"Autauga County, AL",Metro_2023,1.0
198,01001,AL,"Autauga County, AL",Civilian_labor_force_2000,21861.0
199,01001,AL,"Autauga County, AL",Employed_2000,20971.0


In [37]:
# print contents of "Attrubute" Category
df.Attribute.unique()

array(['Rural_Urban_Continuum_Code_2023', 'Urban_Influence_Code_2013',
       'Metro_2023', 'Civilian_labor_force_2000', 'Employed_2000',
       'Unemployed_2000', 'Unemployment_rate_2000',
       'Civilian_labor_force_2001', 'Employed_2001', 'Unemployed_2001',
       'Unemployment_rate_2001', 'Civilian_labor_force_2002',
       'Employed_2002', 'Unemployed_2002', 'Unemployment_rate_2002',
       'Civilian_labor_force_2003', 'Employed_2003', 'Unemployed_2003',
       'Unemployment_rate_2003', 'Civilian_labor_force_2004',
       'Employed_2004', 'Unemployed_2004', 'Unemployment_rate_2004',
       'Civilian_labor_force_2005', 'Employed_2005', 'Unemployed_2005',
       'Unemployment_rate_2005', 'Civilian_labor_force_2006',
       'Employed_2006', 'Unemployed_2006', 'Unemployment_rate_2006',
       'Civilian_labor_force_2007', 'Employed_2007', 'Unemployed_2007',
       'Unemployment_rate_2007', 'Civilian_labor_force_2008',
       'Employed_2008', 'Unemployed_2008', 'Unemployment_rate_2008'

In [38]:
# Select only Unemployment_rate_2023 
df = df[df['Attribute'].isin(['Unemployment_rate_2023', 'Median_Household_Income_2022'])]
df = df.pivot(index='FIPS_Code', columns='Attribute', values='Value')
df.index.names=['fips']

In [39]:
# Rename "Value" to "Unemployment_rate_2023" drop "Attribute" column
# df.rename(columns={"FIPS_Code": "fips"}, inplace=True)
# df.drop(columns=["Attribute", 'Area_Name', "State"], inplace=True)
df.head()


Attribute,Median_Household_Income_2022,Unemployment_rate_2023
fips,,
01001,70148.0,2.2
01003,71704.0,2.3
01005,41151.0,4.4
01007,54309.0,2.5
01009,60553.0,2.1


In [28]:
# Create scores
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
df['IncomeScore'] = scaler.fit_transform(np.log10(df[['Median_Household_Income_2022']]))
df['UnemploymentScore'] = 1-scaler.fit_transform(np.log10(df[['Unemployment_rate_2023']]))

df.head()

Attribute,Median_Household_Income_2022,Unemployment_rate_2023,IncomeScore,UnemploymentScore
fips,,,,
01001,70148.0,2.2,0.503781,0.748440
01003,71704.0,2.3,0.516280,0.732307
01005,41151.0,4.4,0.199922,0.496880
01007,54309.0,2.5,0.357983,0.702046
01009,60553.0,2.1,0.419984,0.765323


In [23]:
# Join housing and unemployment data on fips code
merged = housing.merge(df, on='fips')
merged.head()

,fips,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,...,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,Median_Household_Income_2022,Unemployment_rate_2023,IncomeScore,UnemploymentScore
0,06037,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",06,037,...,920663.626596,919862.912374,915945.298124,908845.790824,903346.252437,898170.722197,82455.0,5.0,0.595872,0.450486
1,17031,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,031,...,320423.157397,321285.261681,322171.942175,322684.457369,322958.701951,322945.461675,76614.0,4.4,0.554014,0.496880
2,48201,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,...,290980.109414,290782.813115,290477.917548,289656.138261,288605.117057,287587.960180,68748.0,4.3,0.492296,0.505224
3,04013,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",04,013,...,479306.556316,478345.760857,476830.653952,474624.466233,472581.818972,470511.495751,83668.0,3.4,0.604192,0.590453
4,06073,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",06,073,...,955819.574596,955928.466256,955387.171534,953232.762654,950029.715441,945139.141643,98365.0,3.9,0.696388,0.540659
